In [89]:
import random
import networkx as nx
import numpy as np
import math
import csv
from sys import maxsize


In [90]:
def gen_prufer(degree_constrained, n):
    result = []
    candidates = list(range(n))
    while len(result) < n:
        # Lấy một số ngẫu nhiên từ danh sách candidates
        x = random.choice(candidates)
        # Nếu số này chưa xuất hiện quá m lần trong danh sách kết quả, thêm số này vào danh sách kết quả
        if result.count(x) < degree_constrained - 1:
            result.append(x)
        # Nếu số này đã xuất hiện quá m lần, loại bỏ số này khỏi danh sách candidates
        if result.count(x) == degree_constrained - 1:
            candidates.remove(x)
    return result

In [91]:
# Đọc dữ liệu từ tệp CSV và lưu trữ nó trong danh sách
def get_distance_table(file_path):
    records = []
    with open('dat.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        # next(reader) # Skip header row
        for row in reader:
            records.append(row)

    # Tạo từ điển để lưu trữ khoảng cách giữa các điểm
    distances_table = {}

    # Tính toán khoảng cách giữa các điểm và lưu trữ kết quả vào từ điển
    for i in range(len(records)):
        for j in range(i + 1, len(records)):
            point1 = (float(records[i][1]), float(records[i][2]))
            point2 = (float(records[j][1]), float(records[j][2]))
            distance = math.dist(point1, point2)
            key = (int(records[i][0]), int(records[j][0]))
            distances_table[key] = distance

    # In ra từ điển khoảng cách
    return distances_table


In [92]:
# distances_table = get_distance_table("")
# a = (1, 0)
# if a in distances_table:
#     print(distances_table[a])
# if a[::-1] in distances_table:
#     print(distances_table[a[::-1]])

In [93]:
def get_distance(edge, distances_table):
    if edge in distances_table:
        return distances_table[edge]
    if edge[::-1] in distances_table:
        return distances_table[edge[::-1]]

In [94]:
def compute_degrees(edges):
    degrees = {}
    for edge in edges:
        for vertex in edge:
            if vertex not in degrees:
                degrees[vertex] = 0
            degrees[vertex] += 1
    return degrees


In [95]:
def calculate_fitness(prufer_sequence, degree_constrained, distances_table):
    tree = nx.from_prufer_sequence(prufer_sequence)
    edges = tree.edges
    degrees = list(compute_degrees(edges).values())
    # print(degrees)

    # Check if the degrees more than target degrees
    if any(x > degree_constrained for x in degrees):
        return 9999999

    # Calculate the sum of the weights of the edges
    cost = 0
    for edge in edges:
        cost = cost + get_distance(edge, distances_table)
    return cost

In [96]:
def crossover(parent1, parent2, crossover_rate=1):
    offspring1 = parent1[:]
    offspring2 = parent2[:]
    if random.random() < crossover_rate:
        crossover_point = random.randint(1, len(parent1) - 2)
        offspring1[crossover_point:], offspring2[crossover_point:] = offspring2[crossover_point:], offspring1[
                                                                                                   crossover_point:]
    return offspring1, offspring2


In [97]:
def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

In [98]:
#
#
# p1 = gen_prufer(degree_constrained, n-2)
# # p2 = gen_prufer(degree_constrained, n-2)
# print(p1)
# #
# print(mutate(p1))

In [99]:
# def run_ga(n, degree_constrained, distances_table, population_size=500, crossover_rate=0.8, mutation_rate=0.1,
#            max_generations=100):
#     population = [gen_prufer(degree_constrained, n - 2) for _ in range(population_size)]
#     # print(population)
#     cost = 999999
#     for generation in range(max_generations):
#         print(population)
#         fitness_values = [calculate_fitness(prufer_sequence, degree_constrained, distances_table) for prufer_sequence in
#                           population]
#         print(fitness_values)
#         new_population = []
#         while len(new_population) < population_size:
#             i, j = np.random.choice(range(population_size), size=2, replace=False,
#                                     p=np.array(fitness_values) / sum(fitness_values))
#             parent1 = population[i]
#             parent2 = population[j]
#             offspring1, offspring2 = crossover(parent1, parent2)
#             new_population.append(offspring1)
#             new_population.append(offspring2)
#         for i in range(len(new_population)):
#             mutate(new_population[i])
#         population = new_population
#     return population

In [100]:
def run_ga(n, degree_constrained, distances_table, population_size=500, crossover_rate=0.8, mutation_rate=0.1,
           max_generations=100):
    population = [gen_prufer(degree_constrained, n - 2) for _ in range(population_size)]
    # print(population)
    fitness_values = [calculate_fitness(prufer_sequence, degree_constrained, distances_table) for prufer_sequence in
                          population]
    counter, generations, min_cost = 0, 0, min(fitness_values)
    best_choice = population[fitness_values.index(min(fitness_values))]

    while counter < max_generations:
        # print(population)
        # print(fitness_values)
        new_population = []
        while len(new_population) < population_size:
            i, j = np.random.choice(range(population_size), size=2, replace=False,
                                    p=np.array(fitness_values) / sum(fitness_values))
            parent1 = population[i]
            parent2 = population[j]
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(offspring1)
            new_population.append(offspring2)
        for i in range(len(new_population)):
            mutate(new_population[i])
        population = new_population
        fitness_values = [calculate_fitness(prufer_sequence, degree_constrained, distances_table) for prufer_sequence in
                          population]
        if min(fitness_values) < min_cost:
            counter, min_cost = 0, min(fitness_values)
            best_choice = population[fitness_values.index(min(fitness_values))]
        else:
            counter += 1
        generations += 1
    return best_choice, generations


In [101]:
degree_constrained = 2
n = 10
distances_table = get_distance_table("")

solution, generation = run_ga(n, degree_constrained, distances_table)
print(solution, generation)
print(calculate_fitness(solution,degree_constrained,distances_table))
# fitness_values = [calculate_fitness(prufer_sequence, degree_constrained, distances_table) for prufer_sequence in
#                           last_generation]
# print(fitness_values)

[4, 7, 5, 3, 6, 2, 1, 0] 100
21.395623132202235
